# Data Prep Kit Demo 1 - Ray Version

This notebook will introduce DPK and showcase some of it's capabilities.

Here is the workflow

![](https://raw.githubusercontent.com/sujee/data-prep-kit-examples/main/media/data-prep-kit-3-workflow.png)

References
- [Data prep kit](https://ibm.github.io/data-prep-kit)  is an open source framework that helps with data wrangling.
- [github repo](https://github.com/IBM/data-prep-kit)


## How to run this notebook

Two options:

- **Option 1 - Google Colab:** easiest option.  no setup required.  Click this link to open this on google colab.  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sujee/data-prep-kit-examples/blob/dev1/dpk-intro/dpk_intro_1_ray.ipynb)
- **Option 2 - Local python dev environment:**  Setup using this guide: [setting up python dev environment](../setup-python-dev-env.md)

The notebook will work as in both environments

## Step-1: Inspect the Data

We will use simple PDFs about Solar system.  The files are [here](https://github.com/sujee/data-prep-kit-examples/tree/main/data/solar-system)

- [earth.pdf](https://github.com/sujee/data-prep-kit-examples/blob/main/data/solar-system/earth.pdf)
- [mars.pdf](https://github.com/sujee/data-prep-kit-examples/blob/main/data/solar-system/mars.pdf)

### How to generate PDFs?

If you like to play around with various inputs files, follow these steps to re-generate PDFs.

**Option 1: markdown -> pdf**

First edit the markdown files using any text editor.

Then use [pandoc](https://pandoc.org/) to convert them to pdfs.

```bash
pandoc  earth.md  -o earth.pdf
pandoc  mars.md  -o mars.pdf
```

**Option 2: Use a word editor or google docs editor**

Write your content and export as PDF

## Step-2: Figure out Runtime Environment

### 2.1 - Determine runtime

Determine if we are running on Google colab or local python environment

In [1]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   RUNNING_IN_COLAB = True
else:
   print("NOT in Colab")
   RUNNING_IN_COLAB = False

NOT in Colab


### 2.2 -Download Data if running on Google Colab

In [2]:
if RUNNING_IN_COLAB:
    !mkdir -p 'input'
    !wget -O 'input/earth.pdf'  'https://raw.githubusercontent.com/sujee/data-prep-kit-examples/main/data/solar-system/earth.pdf'
    !wget -O 'input/mars.pdf'  'https://raw.githubusercontent.com/sujee/data-prep-kit-examples/main/data/solar-system/mars.pdf'
    !wget -O 'input/earth.pdf'  'https://raw.githubusercontent.com/sujee/data-prep-kit-examples/main/data/solar-system/earth.pdf'
    !wget -O 'utils.py'  'https://raw.githubusercontent.com/sujee/data-prep-kit-examples/main/utils.py'

### 2.3 - Install dependencies if running on Google Colab

In [3]:
if RUNNING_IN_COLAB:
    ! pip install  --default-timeout=100  \
        data-prep-toolkit-transforms-ray==0.2.1.dev1 \
        deepsearch-toolkit

### 2.4 - Restart Runtime

After installing dependencies, be sure <font color="red">restart runtime</font>, so libraries will be loaded

You do this by going to **`Runtime --> Restart Session`**

Then you can continue to the next step (no need to re-run the notebook)

## Step-2: Configuration

### 2.1 - Basic Config

In [4]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   RUNNING_IN_COLAB = True
else:
   print("NOT in Colab")
   RUNNING_IN_COLAB = False

NOT in Colab


In [5]:
import os

## Configuration
class MyConfig:
    pass

MY_CONFIG = MyConfig ()

if RUNNING_IN_COLAB:
  MY_CONFIG.INPUT_DATA_DIR = 'input'
else:
  MY_CONFIG.INPUT_DATA_DIR = os.path.join (os.path.abspath (''), '..', 'data', 'solar-system')
MY_CONFIG.OUTPUT_FOLDER = "output"
MY_CONFIG.OUTPUT_FOLDER_FINAL = os.path.join(MY_CONFIG.OUTPUT_FOLDER , "output_final")

## Embedding model
MY_CONFIG.EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

## RAY CONFIGURATION
### For local runs, we can use more parallelism
### For google colab, be conservative

if RUNNING_IN_COLAB:
  MY_CONFIG.RAY_RUNTIME_WORKERS = 2
  MY_CONFIG.RAY_NUM_CPUS =  0.3
  MY_CONFIG.RAY_MEMORY_GB = 2  # GB
else:  # local run
  num_cpus_available =  os.cpu_count()
  # print (num_cpus_available)
  MY_CONFIG.RAY_NUM_CPUS =  1
  MY_CONFIG.RAY_MEMORY_GB = 2  # GB
  # MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
  MY_CONFIG.RAY_RUNTIME_WORKERS = 2

print ('MY_CONFIG.RAY_RUNTIME_WORKERS:', MY_CONFIG.RAY_RUNTIME_WORKERS)
print ('MY_CONFIG.RAY_NUM_CPUS:', MY_CONFIG.RAY_NUM_CPUS)
print ('MY_CONFIG.RAY_MEMORY_GB:', MY_CONFIG.RAY_MEMORY_GB)


MY_CONFIG.RAY_RUNTIME_WORKERS: 2
MY_CONFIG.RAY_NUM_CPUS: 1
MY_CONFIG.RAY_MEMORY_GB: 2


In [6]:
## Add parent dir to path
import os,sys

this_dir = os.path.abspath('')
parent_dir = os.path.dirname(this_dir)
sys.path.append (os.path.abspath (parent_dir))

### 2.2 - Setup input/outpur directories

In [7]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_chunk_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_exact_dedupe_out')
output_docid_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_docid_out')
output_fuzzy_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '05_fuzzy_dedupe_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '06_embeddings_out')

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


## Step-3: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 3.1 - Set Input/output Folder

In [8]:
STAGE = 1

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  output_parquet_dir

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='/home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-examples/dpk-intro/../data/solar-system' --> output='output/01_parquet_out'


### 3.2 - Execute

In [9]:
%%time

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from data_processing_ray.runtime.ray import RayTransformLauncher
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launcher = PythonTransformLauncher(Pdf2ParquetPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")


14:11:46 INFO - Running locally
14:11:46 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': False}
14:11:46 INFO - data factory data_ is using local data access: input_folder - /home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-examples/dpk-intro/../data/solar-system output_folder - output/01_parquet_out
14:11:46 INFO - data factory data_ max_files -1, n_sample -1
14:11:46 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
14:11:46 INFO - pipeline id pipeline_id
14:11:46 INFO - code location None
14:11:46 INFO - number of workers 2 worker options {'num_cpus': 1, 'memory': 2147483648, 'max_restarts': -1}
14:11:46 INFO - actor creation delay 0
14:11:46 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': '

✅ Stage:1 completed successfully
CPU times: user 4.16 s, sys: 1.54 s, total: 5.7 s
Wall time: 44.3 s


### 3.3 - Inspect Generated output

Here we should see one entry per input file processed.

In [10]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (2, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,mars.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf
1,earth.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf



### 3.4 - Understand the output

Here are some interesting attributes to note:

- **filename** : original filename
- **contents** : text
- **document_id**: unique id (UUID) assignd to this document
- **hash** : hash of document
- **pdf_convert_time** : time to convert this pdf in seconds

Let's inspect the **contents** column.  See how the text is being divided up!

In [11]:
import pprint
import json

pprint.pprint (json.loads(output_df.iloc[0, ]['contents']))
# json.loads(output_df.iloc[0, ]['contents'])

{'_name': '',
 'description': {'logs': []},
 'equations': [],
 'figures': [],
 'file-info': {'#-pages': 1,
               'document-hash': '1a83f43f3a202e3f203c1263e36961ecc45d401aad488f638fc5559a584333b2',
               'filename': 'mars.pdf',
               'page-hashes': [{'hash': '551fe7a9bde2a9302f150c0a79a13fcc0868fcf73ac6afb80be645c1174734a0',
                                'model': 'default',
                                'page': 1}]},
 'footnotes': [],
 'main-text': [{'name': 'Section-header',
                'prov': [{'bbox': [133.35, 657.11, 169.63, 667.98],
                          'page': 1,
                          'span': [0, 4]}],
                'text': 'Mars',
                'type': 'subtitle-level-1'},
               {'name': 'Section-header',
                'prov': [{'bbox': [133.1, 630.68, 210.45, 642.34],
                          'page': 1,
                          'span': [0, 12]}],
                'text': 'Solar System',
                'type': 'subtit

In [12]:
pprint.pprint (json.loads(output_df.iloc[1, ]['contents']))

{'_name': '',
 'description': {'logs': []},
 'equations': [],
 'figures': [],
 'file-info': {'#-pages': 1,
               'document-hash': '7401ae81637dbb89e7040dcd5945bbfb75ff8648bb761c69f8a1595e86538748',
               'filename': 'earth.pdf',
               'page-hashes': [{'hash': 'ca802e4bd5a3301792808caea2a47db51f0520888875b77fc230c99ee851c19b',
                                'model': 'default',
                                'page': 1}]},
 'footnotes': [],
 'main-text': [{'name': 'Section-header',
                'prov': [{'bbox': [133.31, 656.9, 173.65, 667.93],
                          'page': 1,
                          'span': [0, 5]}],
                'text': 'Earth',
                'type': 'subtitle-level-1'},
               {'name': 'Section-header',
                'prov': [{'bbox': [133.13, 630.69, 210.44, 642.28],
                          'page': 1,
                          'span': [0, 12]}],
                'text': 'Solar System',
                'type': 'subt

##  Step-4: Doc chunks

In the previous step, we have extracted text from oru PDFs.  But we have the content of entire file as 'one row' in our parquet output.

In this step, we are going to split the documents in chunks, according to their layout segmentation.

This transform uses [Quackling](https://github.com/DS4SD/quackling) `HierarchicalChunker`
to chunk according to the document layout segmentation, i.e. respecting the original document components as paragraphs, tables, enumerations, etc.
It relies on documents converted with the Docling library in the [pdf2parquet transform](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/pdf2parquet/python/README.md) using the option `contents_type: "application/json"`,
which provides the required JSON structure.

### 4.1 - Set Input/output Folder

In [13]:
STAGE = 2

input_folder = output_parquet_dir # previous output folder is the input folder for the current stage
output_folder =  output_chunk_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_chunk_out'


### 4.2 - Execute

In [14]:
%%time

from data_processing_ray.runtime.ray import RayTransformLauncher
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/sujee/apps/anaconda3/envs/data-prep-
[nltk_data]     kit-11-workshop/lib/python3.11/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!
14:12:27 INFO - Running locally
14:12:27 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'output_chunk_column_name': 'contents', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
14:12:27 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_chunk_out
14:12:27 INFO - data factory data_ max_files -1, n_sample -1
14:12:27 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
14:12:27 INFO - pipeline i

✅ Stage:2 completed successfully
CPU times: user 1.37 s, sys: 984 ms, total: 2.36 s
Wall time: 19.4 s


### 4.3 - Inspect Generated output

We would see documents are split into many chunks

In [15]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.head(10)

Files processed : 2
Chunks created : 8
Input data dimensions (rows x columns)=  (2, 12)
Output data dimensions (rows x columns)=  (8, 15)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
0,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.85, 590.24, 478.57, 623.03]"
1,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Solar System\nFor more details about the Solar...,$.main-text[3],1,"[133.19, 570.83, 374.24, 581.07]"
2,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is...",$.main-text[5],1,"[132.87, 500.92, 477.3, 534.56]"
3,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[7],1,"[147.73, 452.7, 476.98, 474.97]"
4,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.87, 590.26, 478.57, 623.05]"
5,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Solar System\nFor more details about our Solar...,$.main-text[3],1,"[133.21, 570.82, 374.81, 581.08]"
6,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.91, 512.46, 477.41, 534.48]"
7,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Earth\n· Distance from the Sun: Average of 149...,$.main-text[7],1,"[147.68, 464.66, 477.03, 487.03]"


### 4.4 - Understanding the Output

Here we see 2 PDF files are split into 6 chunks.  Basically we see the documents are being split along 'natural boundaris' - paragraphs and bullet points

See how **document_id** is carried throughout.  This helps us identify original documents.

Also note **contents** is now plain text (not JSON as before)

In [16]:
output_df[['filename', 'contents']]

,filename,contents
0,mars.pdf,Solar System\nOur solar system is a vast and f...
1,mars.pdf,Solar System\nFor more details about the Solar...
2,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is..."
3,mars.pdf,Basic facts about Mars:\n· Distance from the S...
4,earth.pdf,Solar System\nOur solar system is a vast and f...
5,earth.pdf,Solar System\nFor more details about our Solar...
6,earth.pdf,Earth\nEarth is the third planet from the Sun....
7,earth.pdf,Earth\n· Distance from the Sun: Average of 149...


In [17]:
for f in output_df['filename'].unique():
    print ('==========' , f, '===========')
    chunks = output_df[output_df['filename'] == f]['contents']
    for idx , chunk in enumerate(chunks):
        print (f'-------Chunk {idx}------\n{chunk}\n-------')

========== mars.pdf ===========
-------Chunk 0------
Solar System
Our solar system is a vast and fascinating expanse, comprising eight planets, five dwarf planets, numerous moons, asteroids, comets, and other celestial bodies. At its center lies the star we call the Sun.
-------
-------Chunk 1------
Solar System
For more details about the Solar system see Chapter 1.
-------
-------Chunk 2------
Mars
Mars, the fourth planet from the Sun, is a cold, desert world with a thin atmosphere composed primarily of carbon dioxide. Its reddish hue comes from iron oxide, or rust, prevalent on its surface.
-------
-------Chunk 3------
Basic facts about Mars:
· Distance from the Sun: Average of 228 million kilometers (142 million miles)
· Rotation Period: 24.6 hours (one Martian day - called a "sol")
· Moons: Two small moons, Phobos and Deimos.
-------
========== earth.pdf ===========
-------Chunk 0------
Solar System
Our solar system is a vast and fascinating expanse, comprising eight planets, five 

## Step-5: Exact Dedup



### 5.1 - Set Input/output Folder

In [18]:
STAGE  = 3

input_folder = output_chunk_dir # previous output folder is the input folder for the current stage
output_folder =  output_exact_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output/02_chunk_out' --> output='output/03_exact_dedupe_out'


### 5.2 - Execute

In [19]:
%%time

from data_processing_ray.runtime.ray import RayTransformLauncher
from ededup_transform_ray import EdedupRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(EdedupRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

14:12:44 INFO - Running locally
14:12:44 INFO - exact dedup params are {'doc_column': 'contents', 'hash_cpu': 0.5, 'num_hashes': 2}
14:12:44 INFO - data factory data_ is using local data access: input_folder - output/02_chunk_out output_folder - output/03_exact_dedupe_out
14:12:44 INFO - data factory data_ max_files -1, n_sample -1
14:12:44 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
14:12:44 INFO - pipeline id pipeline_id
14:12:44 INFO - code location None
14:12:44 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
14:12:44 INFO - actor creation delay 0
14:12:44 INFO - job details {'job category': 'preprocessing', 'job name': 'ededup', 'job type': 'ray', 'job id': 'job_id'}
2024-09-18 14:12:46,665	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(orchestrate pid=917432) 14:12:47 INFO - orchestra

✅ Stage:3 completed successfully
CPU times: user 136 ms, sys: 157 ms, total: 293 ms
Wall time: 15.1 s


### 5.3 - Inspect Generated output

In [20]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.head(10)

Input data dimensions (rows x columns)=  (8, 15)
Output data dimensions (rows x columns)=  (7, 15)
Input chunks before exact dedupe : 8
Output chunks after exact dedupe : 7
Duplicate chunks removed :   1


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
0,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.85, 590.24, 478.57, 623.03]"
1,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Solar System\nFor more details about the Solar...,$.main-text[3],1,"[133.19, 570.83, 374.24, 581.07]"
2,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is...",$.main-text[5],1,"[132.87, 500.92, 477.3, 534.56]"
3,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[7],1,"[147.73, 452.7, 476.98, 474.97]"
4,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Solar System\nFor more details about our Solar...,$.main-text[3],1,"[133.21, 570.82, 374.81, 581.08]"
5,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.91, 512.46, 477.41, 534.48]"
6,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Earth\n· Distance from the Sun: Average of 149...,$.main-text[7],1,"[147.68, 464.66, 477.03, 487.03]"


In [21]:
output_df[['filename', 'contents']]

,filename,contents
0,mars.pdf,Solar System\nOur solar system is a vast and f...
1,mars.pdf,Solar System\nFor more details about the Solar...
2,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is..."
3,mars.pdf,Basic facts about Mars:\n· Distance from the S...
4,earth.pdf,Solar System\nFor more details about our Solar...
5,earth.pdf,Earth\nEarth is the third planet from the Sun....
6,earth.pdf,Earth\n· Distance from the Sun: Average of 149...


In [22]:
for f in output_df['filename'].unique():
    print ('==========' , f, '===========')
    chunks = output_df[output_df['filename'] == f]['contents']
    for idx , chunk in enumerate(chunks):
        print (f'-------Chunk {idx}------\n{chunk}\n-------')

========== mars.pdf ===========
-------Chunk 0------
Solar System
Our solar system is a vast and fascinating expanse, comprising eight planets, five dwarf planets, numerous moons, asteroids, comets, and other celestial bodies. At its center lies the star we call the Sun.
-------
-------Chunk 1------
Solar System
For more details about the Solar system see Chapter 1.
-------
-------Chunk 2------
Mars
Mars, the fourth planet from the Sun, is a cold, desert world with a thin atmosphere composed primarily of carbon dioxide. Its reddish hue comes from iron oxide, or rust, prevalent on its surface.
-------
-------Chunk 3------
Basic facts about Mars:
· Distance from the Sun: Average of 228 million kilometers (142 million miles)
· Rotation Period: 24.6 hours (one Martian day - called a "sol")
· Moons: Two small moons, Phobos and Deimos.
-------
========== earth.pdf ===========
-------Chunk 0------
Solar System
For more details about our Solar system see Chapter 1.
-------
-------Chunk 1------

### 5.4 - Understanding the output

Remember we had 8 chunks initially.  Now we have 7!  One duplicate chunk is removed.

If you look at the PDF, the following common paragraph in `earth.pdf` and `mars.pdf`  is removed from one of the documents!  Pretty neat, eh!

```text
## Solar System

Our solar system is a vast and fascinating expanse, comprising eight planets, five dwarf planets, numerous moons, asteroids, comets, and other celestial bodies. At its center lies the star we call the Sun.
```

## Step-6:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set **hash_column** to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set **int_id_column** to the name of the column, where you want to store it.

**This is a pre-requisite for fuzzy dedup** in the pipeline.

### 6.1 - Set Input/output Folder

In [23]:

# Input for this stage is the output of exact dedeup component
# output of this component makes it possible for fdedup component to run on data.

STAGE  = 4

input_folder = output_exact_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_docid_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output/03_exact_dedupe_out' --> output='output/04_docid_out'


### 6.2 - Execute

In [24]:
%%time

from data_processing_ray.runtime.ray import RayTransformLauncher
from doc_id_transform_ray import DocIDRayTransformConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "hash_column",
    "doc_id_int_column": "int_id_column",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(DocIDRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

14:12:59 INFO - Running locally
14:12:59 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'hash_column', 'int_column': 'int_id_column'}
14:12:59 INFO - data factory data_ is using local data access: input_folder - output/03_exact_dedupe_out output_folder - output/04_docid_out
14:12:59 INFO - data factory data_ max_files -1, n_sample -1
14:12:59 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
14:12:59 INFO - pipeline id pipeline_id
14:12:59 INFO - code location None
14:12:59 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
14:12:59 INFO - actor creation delay 0
14:12:59 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_id', 'job type': 'ray', 'job id': 'job_id'}
2024-09-18 14:13:01,844	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(orchestrate pid=91908

✅ Stage:4 completed successfully
CPU times: user 130 ms, sys: 160 ms, total: 290 ms
Wall time: 15.1 s


### 6.3 - Inspect Generated output

You will notice we have two extra columns

- **hash_column**
- **int_id_column**

But still the same number or rows as before

In [25]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.head(10)

Input data dimensions (rows x columns)=  (7, 15)
Output data dimensions (rows x columns)=  (7, 17)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,hash_column,int_id_column
0,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.85, 590.24, 478.57, 623.03]",44c6e373258c7cdc03f75a8e96a9b160f9aa4e4baf5674...,1
1,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Solar System\nFor more details about the Solar...,$.main-text[3],1,"[133.19, 570.83, 374.24, 581.07]",dee4c03474c98efdabbadbcc4ce91138c7820f4ac8ff07...,2
2,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is...",$.main-text[5],1,"[132.87, 500.92, 477.3, 534.56]",a31663e06fac41470ecc459f5a58658a3f9997d7801053...,3
3,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[7],1,"[147.73, 452.7, 476.98, 474.97]",7ff317954ec5f3b15607c053c30c2b0db0f6b64cc3295a...,4
4,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Solar System\nFor more details about our Solar...,$.main-text[3],1,"[133.21, 570.82, 374.81, 581.08]",d7be13d7dee96cf2384072d0eb01981e0e75eec2e7bc6d...,5
5,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.91, 512.46, 477.41, 534.48]",7c4a750e2215f231803a6f8078bde1e9699034fb033dd3...,6
6,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Earth\n· Distance from the Sun: Average of 149...,$.main-text[7],1,"[147.68, 464.66, 477.03, 487.03]",0337be5c940b3df316e391ae3dc20379442257498812e4...,7


## Step-7: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with the goal of removing code files that may have **slight variations** and thereby unbiasing
the data further.

Small variations are quite commonly seen in code data in the form of variations in the values of variables, addittion of logging statements etc.

### 7.1 - Set Input/output Folder

In [26]:
## Input to this component is the output of doc_id generator component.

STAGE  = 5

input_folder = output_docid_dir # previous output folder is the input folder for the current stage
output_folder =  output_fuzzy_dedupe_dir
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output/04_docid_out' --> output='output/05_fuzzy_dedupe_out'


### 7.2 - Execute

In [27]:
%%time

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration
from data_processing_ray.runtime.ray import RayTransformLauncher

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "int_id_column",
    "fdedup_cluster_column": "hash_column",
    # infrastructure
    "fdedup_bucket_cpu": 0.3,
    "fdedup_doc_cpu": 0.3,
    "fdedup_mhash_cpu": 0.3,
    "fdedup_num_doc_actors": 1,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 1,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.7, # (default 0.8)
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

14:13:15 INFO - Running locally
14:13:15 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'int_id_column', 'cluster_column': 'hash_column', 'bucket_cpu': 0.3, 'mhash_cpu': 0.3, 'doc_cpu': 0.3, 'num_doc_actors': 1, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 1, 'num_permutations': 64, 'threshold': 0.7, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
14:13:15 INFO - data factory data_ is using local data access: input_folder - output/04_docid_out output_folder - output/05_fuzzy_dedupe_out
14:13:15 INFO - data factory data_ max_files -1, n_sample -1
14:13:15 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
14:13:15 INFO - pipeline id pipeline_id
14:13:15 INFO - code location None
14:13:15 INFO - number of

✅ Stage:5 completed successfully
CPU times: user 199 ms, sys: 189 ms, total: 388 ms
Wall time: 32.5 s


### 7.3 - Inspect Generated output

In [28]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.head(10)

Input data dimensions (rows x columns)=  (7, 15)
Output data dimensions (rows x columns)=  (6, 17)
Duplicate chunks removed  by fuzzy-dedupe:   1


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,int_id_column,hash_column
0,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.85, 590.24, 478.57, 623.03]",1,-1
1,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Solar System\nFor more details about the Solar...,$.main-text[3],1,"[133.19, 570.83, 374.24, 581.07]",2,5
2,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is...",$.main-text[5],1,"[132.87, 500.92, 477.3, 534.56]",3,-1
3,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[7],1,"[147.73, 452.7, 476.98, 474.97]",4,-1
4,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.91, 512.46, 477.41, 534.48]",6,-1
5,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Earth\n· Distance from the Sun: Average of 149...,$.main-text[7],1,"[147.68, 464.66, 477.03, 487.03]",7,-1


In [29]:
output_df[['filename', 'contents']]

,filename,contents
0,mars.pdf,Solar System\nOur solar system is a vast and f...
1,mars.pdf,Solar System\nFor more details about the Solar...
2,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is..."
3,mars.pdf,Basic facts about Mars:\n· Distance from the S...
4,earth.pdf,Earth\nEarth is the third planet from the Sun....
5,earth.pdf,Earth\n· Distance from the Sun: Average of 149...


In [30]:
for f in output_df['filename'].unique():
    print ('==========' , f, '===========')
    chunks = output_df[output_df['filename'] == f]['contents']
    for idx , chunk in enumerate(chunks):
        print (f'-------Chunk {idx}------\n{chunk}\n-------')

========== mars.pdf ===========
-------Chunk 0------
Solar System
Our solar system is a vast and fascinating expanse, comprising eight planets, five dwarf planets, numerous moons, asteroids, comets, and other celestial bodies. At its center lies the star we call the Sun.
-------
-------Chunk 1------
Solar System
For more details about the Solar system see Chapter 1.
-------
-------Chunk 2------
Mars
Mars, the fourth planet from the Sun, is a cold, desert world with a thin atmosphere composed primarily of carbon dioxide. Its reddish hue comes from iron oxide, or rust, prevalent on its surface.
-------
-------Chunk 3------
Basic facts about Mars:
· Distance from the Sun: Average of 228 million kilometers (142 million miles)
· Rotation Period: 24.6 hours (one Martian day - called a "sol")
· Moons: Two small moons, Phobos and Deimos.
-------
========== earth.pdf ===========
-------Chunk 0------
Earth
Earth is the third planet from the Sun. It's our home planet. Earth is the only place we k

### 7.4- Understanding the output

So we started with 7 rows and ended up with 6.  Fuzzy dedupe removed the following **very similar** chunk.

These are pretty similar chunks except for the words 'the' and 'our'

**earth.pdf**

`For more details about *our* Solar system see Chapter 1.`

**mars.pdf**

`For more details about *the* Solar system see Chapter 1.`

Pretty neat, eh? 👏

### Configuring Fuzzy de-dupe

You can tweak fuzzy dedupe by tweaking the following parameters

```python
# fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.7, #  (default 0.8)
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
```

In our case, we set `fdedup_threshold` parameter to 0.7.  


## Step-8:   Text encoding

Encode text for the vector storage.

### 8.1 - Set Input/output Folder

In [31]:
STAGE  = 6

input_folder = output_fuzzy_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_embeddings_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output/05_fuzzy_dedupe_out' --> output='output/06_embeddings_out'


### 8.2 - Execute

In [32]:
%%time

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

14:13:48 INFO - Running locally
14:13:48 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
14:13:48 INFO - data factory data_ is using local data access: input_folder - output/05_fuzzy_dedupe_out output_folder - output/06_embeddings_out
14:13:48 INFO - data factory data_ max_files -1, n_sample -1
14:13:48 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
14:13:48 INFO - pipeline id pipeline_id
14:13:48 INFO - code location None
14:13:48 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
14:13:48 INFO - actor creation delay 0
14:13:48 INFO - job details {'job category': 'preprocessing', 'job name': 'text_encoder', 'job type': 'ray', 'job id': 'job_id'}
2024-09-18 14:13:50,307	INFO worker.py:1744 -- Started a local Ray instance.

✅ Stage:6 completed successfully
CPU times: user 625 ms, sys: 273 ms, total: 898 ms
Wall time: 22.2 s


### 8.3 - Inspect Generated output

You will see a column called `embeddings` added at the end.  This the text content converted into vectors or embeddings.  We used the model `sentence-transformers/all-MiniLM-L6-v2`

In [33]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.head(10)

Input data dimensions (rows x columns)=  (6, 17)
Output data dimensions (rows x columns)=  (6, 18)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,int_id_column,hash_column,embeddings
0,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Solar System\nOur solar system is a vast and f...,$.main-text[2],1,"[132.85, 590.24, 478.57, 623.03]",1,-1,"[0.0077404897, -0.020559434, 0.026426662, 0.01..."
1,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Solar System\nFor more details about the Solar...,$.main-text[3],1,"[133.19, 570.83, 374.24, 581.07]",2,5,"[-0.051861413, 0.0035226392, 0.030617053, 0.04..."
2,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,"Mars\nMars, the fourth planet from the Sun, is...",$.main-text[5],1,"[132.87, 500.92, 477.3, 534.56]",3,-1,"[0.07728298, 0.024971062, -0.04318075, 0.05809..."
3,mars.pdf,1,0,11,e098afda-7d0a-41f3-9bca-3003820a1dc5,pdf,6dfaeed444efc80bf32e0c67308a57c38cc8d27a6405ec...,2531,2024-09-18T14:12:13.777537,1.681353,mars.pdf,Basic facts about Mars:\n· Distance from the S...,$.main-text[7],1,"[147.73, 452.7, 476.98, 474.97]",4,-1,"[0.1059802, 0.025460616, 0.02362733, 0.0390564..."
4,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Earth\nEarth is the third planet from the Sun....,$.main-text[5],1,"[132.91, 512.46, 477.41, 534.48]",6,-1,"[0.0724358, -0.058001805, -0.01977186, -0.0243..."
5,earth.pdf,1,0,11,3e0d8dfe-fc10-48d0-a803-b886bb5702f4,pdf,0260c5d12de8d1ce27a6c096f7983336b1ded574ad3ab8...,2407,2024-09-18T14:12:13.589200,1.766808,earth.pdf,Earth\n· Distance from the Sun: Average of 149...,$.main-text[7],1,"[147.68, 464.66, 477.03, 487.03]",7,-1,"[0.06319313, 0.05406263, 0.042939227, 0.002394..."


## Step-9: Copy output to final output dir

In [34]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/06_embeddings_out' --> 'output/output_final'
